In [6]:

import random
import math
import numpy as np
import pandas as pd
from sklearn import cluster, datasets
from sklearn.metrics import accuracy_score
from sklearn import preprocessing


# Data Preprocessing

## Abalone dataset

In [7]:
#data_x : features (Dataframe)
#data_y : labels   (np.array)
'''
column_names = ["sex", "length", "diameter", "height", "whole weight", 
                "shucked weight", "viscera weight", "shell weight", "rings"]
abalone = pd.read_csv("abalone.data",names=column_names)

data_x = abalone.drop(columns=["rings"])
dfDummies = pd.get_dummies(data_x['sex'], prefix = 'sex')
data_x = pd.concat([data_x.drop(columns=['sex']), dfDummies], axis=1)

data_y = np.array(abalone["rings"])
data_y[data_y<11]    = 0
data_y[data_y>=11]   = 1
Simulate_time = 50
'''

'\ncolumn_names = ["sex", "length", "diameter", "height", "whole weight", \n                "shucked weight", "viscera weight", "shell weight", "rings"]\nabalone = pd.read_csv("abalone.data",names=column_names)\n\ndata_x = abalone.drop(columns=["rings"])\ndfDummies = pd.get_dummies(data_x[\'sex\'], prefix = \'sex\')\ndata_x = pd.concat([data_x.drop(columns=[\'sex\']), dfDummies], axis=1)\n\ndata_y = np.array(abalone["rings"])\ndata_y[data_y<11]    = 0\ndata_y[data_y>=11]   = 1\nSimulate_time = 50\n'

## Iris Dataset

In [21]:
#data_x : features (Dataframe)
#data_y : labels   (np.array)

iris = datasets.load_iris()
data_x = pd.DataFrame(iris.data)

data_y = iris.target

Simulate_time = 10


In [105]:
class our_k_means:
    def __init__(self,data_x,data_y,m = 2,epsilon = 1.0e-6):
        self.data_x = pd.DataFrame(preprocessing.scale(data_x))
        self.data_y = data_y
        self.label  = pd.DataFrame(data_y)
        self.acc    = 0
        
        #資料特性
        self.DCNT = len(self.data_x)               #資料個數
        self.DIM  = len(self.data_x.columns)       #資料維度
        self.K    = len(np.unique(data_y))         #叢聚個數
        #self.K    = np.amax(self.data_y)+1        #叢聚個數
        self.MAX_ITER = 100                        #最大迭代 
        self.MIN_PT = 0                            #最小變動點
        
        #k-means過程的參數
        self.m    = m                       #m              :hyper parameter,控制fuzzy程度的變數,通常為2
        self.epsilon = epsilon              #epsilon        :收斂的閾值
        self.data =[]                       #data[DCNT][DIM]:資料
        self.cent =[]                       #cent[K][DIM]   :各centroid的座標
        self.table=[]                       #table[DCNT][K] :各資料對各cluster的membership values matrix
        self.dis_k=[]                       #dis_k[K][DIM]  :各cluster的座標和
        self.cent_c=[]                      #cent_c[K]      :各cluster的擁有資料數和
        self.nearest=[]                     #nearest[DNST]  :各資料最可能屬於的cluster
        self.iterl = 0 
        self.obj_value = 0
        self.prev_obj_value = 0
        
        #計算acc時的參數
        self.origin_mass = []
        self.cent_name = []

        
    #run k-means    
    def run(self):
        
        #initialize tables
        self.kmeans_init()   #初始化centroid
                
        #first iteration 
        self.iterl = 0
        self.update_table()
        self.obj_value = self.cal_obj_func()
        self.prev_obj_value = self.obj_value*2
        
        #update centroid & data clustering
        while self.iterl<self.MAX_ITER and abs(self.prev_obj_value-self.obj_value)>=self.epsilon :
            self.prev_obj_value = self.obj_value
            self.iterl+=1
            self.update_cent()
            self.update_table()
            self.obj_value = self.cal_obj_func()
        
        #self.print_result()    
        
    #Calculate average accuracy    
    def calculate_acc(self,iterate_times):
        self.acc = 0
        i = 0
        while( i < iterate_times):
            self.run()
            self.calculate_origin_mass()
            self.cent_name = self.centroid_names()
            # Avoid the rare situations that some cluster are gone
            #if len(np.unique(self.cent_name)) != self.K:
            #    continue
                
            self.nearest_cluster()
            i += 1
            self.acc += accuracy_score(self.data_y,self.nearest)
            
            #self.print_result()
        
        if iterate_times is not 0:
            self.acc /= iterate_times
        
        print("Average accuracy for ",iterate_times," times : ",self.acc)
        return self.acc
#---------------------------------------------------------------------------------
#----------------Subfunctions of calculate_acc(iterate_times)---------------------
#---------------------------------------------------------------------------------
    def centroid_names(self):
        cent_name = np.zeros(self.K)
        
        for i in range(self.K):
            min_dist=float("inf")
            name = 0
            for j in range(self.K):
                dist = np.linalg.norm(self.cent[i] - self.origin_mass[j])
                if dist < min_dist:
                    min_dist = dist
                    name = j
            cent_name[i] = name
            
        return cent_name
    
    def calculate_origin_mass(self):
        self.origin_mass = np.zeros((self.K,self.DIM))
        
        counter = np.zeros(self.K)
        for i in range(self.K):
            counter[i] = len(self.data_y[self.data_y==i])
            
        
        for j in range(self.DIM):
            for i in range(self.DCNT):
                a = self.data_y[i]
                self.origin_mass[a][j] += self.data_x.iloc[i,j]
            for i in range(self.K):  
                if counter[i] is not 0:
                    self.origin_mass[i][j] /= counter[i]
    
    def nearest_cluster(self):
        self.nearest = np.zeros(self.DCNT)
        
        for i in range(len(self.data_x.index)):
            self.nearest[i] = self.cent_name[np.argmax(self.table[i])]
        
#---------------------------------------------------------------------------------
#------------------------------Subfunctions of run()------------------------------
#---------------------------------------------------------------------------------    
    def kmeans_init(self):
        
        self.data = self.data_x.values
        self.cent = np.zeros((self.K,self.DIM))
        self.table= np.zeros((self.DCNT,self.K))
        self.dis_k= np.zeros((self.K,self.DIM))
        self.cent_c=np.zeros(self.K)
        self.U    = np.zeros((self.DCNT,self.K))
                
        pick = []
        counter = 0
        while(counter<self.K):
            rnd = random.randint(0,self.DCNT-1)
            if(rnd not in pick):
                pick.append(rnd)
                counter=counter+1
                
        for i in range(self.K):
            for j in range(self.DIM):
                self.cent[i][j] = self.data[pick[i]][j] 
      

    def update_cent(self):
        
        for k in range(self.K):
            down = 0
            for i in range(self.DCNT):
                down += self.table[i][k]
                
            for i in range(self.DCNT):    
                for j in range(self.DIM):
                    self.cent[k][j] += self.data_x.iloc[i,j]*self.table[i][k]
                    
            for j in range(self.DIM):
                self.cent[k][j] /= down
                    
    def cal_w(self,i,j):
        w = 0
        dis = np.linalg.norm(self.data_x.iloc[i].values-self.cent[j])
        for c in range(self.K):
            dis_c = np.linalg.norm(self.data_x.iloc[i].values-self.cent[c])
            if dis_c != 0:
                w += math.pow((dis/dis_c),2/(self.m-1))
        
        if(w != 0):
            w = 1/w
            
        return w
            
    def update_table(self):
        for i in range(self.DCNT):
            for j in range(self.K):
                self.table[i][j] = self.cal_w(i,j)
                
    def cal_obj_func(self):
        obj_value = 0
        for i in range(self.DCNT):
            for j in range(self.K):
                obj_value += self.table[i][j]*math.pow(np.linalg.norm(self.data_x.iloc[i].values-self.cent[j]),2)
        return obj_value

    def print_cent(self):
        print("Centroids:")
        print(self.cent)

    def print_result(self):
        print("K means:")
        print(self.table)
        print("Object function value = ",end='')
        print(self.obj_value)
        print("Previous Object function value = ",end='')
        print(self.prev_obj_value)
        print("iter = ",end='')
        print(self.iterl)     
    

## Here comes our FCM
### Let's run it!

In [106]:
result = our_k_means(data_x,data_y)
result.calculate_acc(Simulate_time)

Average accuracy for  10  times :  0.8600000000000001


0.8600000000000001

In [107]:
result.print_cent()

Centroids:
[[ 0.63745732 -0.19331011  0.66769739  0.67797323]
 [-0.85321793  0.64599073 -1.0692856  -1.03168927]
 [ 0.21615645 -0.45328001  0.40222018  0.35429083]]


In [108]:
result.print_result()

K means:
[[0.02359739 0.94615959 0.03024302]
 [0.06370665 0.84468916 0.09160419]
 [0.0409838  0.90289018 0.05612602]
 [0.05880258 0.85886985 0.08232757]
 [0.03882277 0.91257092 0.04860631]
 [0.11064488 0.76268566 0.12666945]
 [0.03797323 0.9119979  0.05002886]
 [0.01471581 0.96584571 0.01943849]
 [0.1066532  0.74230182 0.15104498]
 [0.04624529 0.88875944 0.06499527]
 [0.06583006 0.85491067 0.07925927]
 [0.02296243 0.94664819 0.03038937]
 [0.06970171 0.83140216 0.09889613]
 [0.09881419 0.76589885 0.13528696]
 [0.15023999 0.68414839 0.16561162]
 [0.2060267  0.5795843  0.214389  ]
 [0.10924651 0.76559278 0.12516071]
 [0.02003322 0.9542151  0.02575167]
 [0.11124648 0.76076754 0.12798598]
 [0.07565346 0.83423497 0.09011157]
 [0.02071033 0.95204574 0.02724393]
 [0.05458188 0.87879737 0.06662074]
 [0.06162938 0.86143259 0.07693803]
 [0.00361595 0.99135113 0.00503291]
 [0.02221065 0.9481902  0.02959915]
 [0.06089029 0.85032331 0.0887864 ]
 [0.00706487 0.98349319 0.00944195]
 [0.02281321 0.9479